In [1]:
import re
import string
from nltk.cluster.util import cosine_distance
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import nltk
nltk.download('wordnet') # first-time use only

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mtool\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## NY Times Best Seller List
I chose 24 titles off the New York times Best Seller list. I then searched for the 1st and the 20th organic search results for the first book in the list and used those search results for the second question.

In [2]:

AmazonBookList =  ['All the Ugly and Wonderful Things: A Novel',
                   'The Tuscan Child',
                   'Where the Crawdads Sing',
                   'The Nightingale: A Novel',
                   'The Goldfinch: A Novel (Pulitzer Prize for Fiction)',
                   'The Life We Bury',
                   'All the Light We Cannot See: A Novel',
                   'Spilled Milk: Based on a true story',
                   'What Alice Forgot',
                   'The Flight Attendant: A Novel',
                   'Winter Garden',
                   "The Storyteller's Secret: A Novel",
                   'Ordinary Grace: A Novel',
                   'All the Ugly and Wonderful Things: A Novel',
                   'It Ends with Us: A Novel',
                   'The Shack: Where Tragedy Confronts Eternity',
                   'Beneath a Scarlet Sky: A Novel',
                   'Before We Were Yours: A Novel',
                   'Small Great Things: A Novel',
                   "The Boy on the Wooden Box: How the Impossible Became Possible . . . on Schindler's List",
                   'A Man Called Ove: A Novel',
                   "The Ladies' Room",
                   'The Butterfly Garden (The Collector Book 1)',
                   'HOSTILE WITNESS: A Josie Bates Thriller (The Witness Series Book 1)']

stopword_list = ['a','the','and','novel','s']

GoogleResults = ["All the Ugly and Wonderful Things: A Novel","All the Ugly and Wonderful Things by Bryn Greenwood - Goodreads\
All the Ugly and Wonderful Things book. Read 9840 reviews from the world's largest community for readers.\
As the daughter of a meth dealer, Wavy knows n..","All The Ugly And Wonderful Things - By Bryn Greenwood (Hardcover\
A New York Times and USA Today bestseller - Book of the Month Club 2016 Book of the Year - Second Place Goodreads Best Fiction\
of 2016. A beautiful and" ]

#### Code found online at to calculate cosine similarity from:
https://sites.temple.edu/tudsc/2017/03/30/measuring-similarity-between-texts-in-python/ 

##### Helper functions

In [3]:

lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
     return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
     return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopword_list)
    
def cos_similarity(textlist):
    tfidf = TfidfVec.fit_transform(textlist)
    return (tfidf * tfidf.T).toarray()

## Question 1 - Find the two titles that are most similar
#### Calculate similarity matrix
The matrix below has rows for each book, and the cosine similarity value for all books in the corpus. The matrix is arranged by *[book:book]* For example *[Row0:Col0]* = 1.  This is the cosine similarity for Book 1 against itself. If we look at *[Row0:Col6]*, the cosine similarity value for Book 1 and Book 7 is .16600436. We search this matrix to find the largest value that is not 1 and that corresponds to the books that are most similar.  Each value of 0 indicates titles that have no similarity, and in this list there are many books with no similarity.

In [4]:
titleMatrix = cos_similarity(AmazonBookList)
print (titleMatrix)

[[1.         0.         0.         0.         0.         0.
  0.16600436 0.         0.         0.         0.         0.
  0.         1.         0.         0.         0.         0.
  0.23476561 0.         0.         0.         0.         0.        ]
 [0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         1.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.21465975 0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.

**In the code belos we sort the array to find the largest value that is not 1. We'll then search the original matrix to find the *[Row:Col]* that corresponds to that number and those indicies will be the matching books.**

In [5]:
titleArray = np.asarray(titleMatrix).reshape(-1)
#titleArray = np.unique(titleArray.sort())
uniquArray = np.unique(titleArray)
print ('Unique Cosine Similarity Scores')
print('-------------------------------')
print (uniquArray)
highScore = 0
lowScore = 1
for score in uniquArray:
    if (score > highScore) and (score < .99 ): # Use .99 for rounding errors in cosine
        highScore = score
#    if (score < lowScore) and (score >0):
#        lowScore = score
print('\nHighScore')
print('--------')
print(highScore)
    

Unique Cosine Similarity Scores
-------------------------------
[0.         0.14696901 0.16600436 0.17252475 0.18700265 0.20784557
 0.21465975 0.23105876 0.23476561 0.28122516 1.         1.
 1.         1.        ]

HighScore
--------
0.2812251571902398


**Now find the index with the high score and find the names of the books.**

In [6]:
book1, book2 = np.where(titleMatrix == highScore)
print(book1,book2)

[10 22] [22 10]


In [7]:
print('{} at index {:2d} is most similar to {} at index {:2d} with a cosine similarity of {:1.5f}'.format(
    AmazonBookList[book1[0]], book1[0], AmazonBookList[book1[1]], book1[1],highScore))

Winter Garden at index 10 is most similar to The Butterfly Garden (The Collector Book 1) at index 22 with a cosine similarity of 0.28123


## Question 2 
#### Compare the two capsules from organic search with the book title

In [8]:
searchResultsMatrix = cos_similarity(GoogleResults)
print (searchResultsMatrix)

[[1.         0.42708791 0.22420408]
 [0.42708791 1.         0.25452838]
 [0.22420408 0.25452838 1.        ]]


In [9]:
searchArray = np.asarray(searchResultsMatrix).reshape(-1)
#titleArray = np.unique(titleArray.sort())
uniqueSearchArray = np.unique(searchArray)
print ('Unique Cosine Similarity Scores')
print('-------------------------------')
print (uniqueSearchArray)
highScore = 0
lowScore = 1
for score in uniqueSearchArray:
    if (score > highScore) and (score < .99 ): # Use .99 for rounding errors in cosine
        highScore = score
#    if (score < lowScore) and (score >0):
#        lowScore = score
print('\nHighScore')
print('--------')
print(highScore)
    

Unique Cosine Similarity Scores
-------------------------------
[0.22420408 0.25452838 0.42708791 1.         1.         1.        ]

HighScore
--------
0.4270879126473651


In [10]:
text1, text2 = np.where(searchResultsMatrix == highScore)
print(text1,text2)

[0 1] [1 0]


In [11]:
print('The text at index {} has most in common with the title with a cosine similarity score of {:1.5f}'.format(
text1[1],highScore))

The text at index 1 has most in common with the title with a cosine similarity score of 0.42709


### Discussion of findings
The pairwise cosine comparison scores of the first 24 titles in the New York Times Best Seller list was not very high for most titles. The results of this comparison will change as the titles change, but this result is expected. Most of the titles are short and do not contain any words in common. The titles that had the largest similarity score both had the word *garden* in them so this result makes sense. Many of the titles copied from the Amazon website had a description of *A Novel* added to the end of the title name. This description is not part of the original title so the word *Novel* was cleaned from the text. If this word had been retained we no doubtedly would have seen a larger collection of non-zero scores.

When comparing the 1st and the 20th organic web results, the first web result had a higher similariy score. This makes sense as the Google search algorithm should list hits with a higher similarity score before those with a lower similarity score. We can speculate that if we looked at all 20 organic results, the similarity scores would go down as we move down the list of search results. This assumes our similarity scoring is similar to the one used by Google.
